# Import modules and set core NRPy+ parameters

## Import modules

In [1]:
from outputC import lhrh, outCfunction, outC_function_dict
import finite_difference as fin
import NRPy_param_funcs as par
import grid as gri
import indexedexp as ixp
import reference_metric as rfm
import cmdline_helper as cmd
import sympy as sp
import shutil, os, sys, time
import pickle

## Set C codes directory variables

In [2]:
# Create C code output directory
Ccodesdir = os.path.join("BSSN_LinearScalarFieldEvolution_Ccodes/")
# Remove C code output directory if it exists
shutil.rmtree(Ccodesdir, ignore_errors=True)
# Create a fresh directory
cmd.mkdir(Ccodesdir)

# Create executable output directory
outdir = os.path.join(Ccodesdir, "output/")
cmd.mkdir(outdir)

## Set core NRPy+ parameters

In [3]:
# Set spatial dimension to 3
par.set_parval_from_str("grid::DIM", 3)
DIM = par.parval_from_str("grid::DIM")

# Enable/Disable SIMD-optimised code
SIMD_enable = True

# If SIMD is enabled, copy SIMD_intrinsics.h to the C codes directory
cmd.mkdir(os.path.join(Ccodesdir, "SIMD/"))
if SIMD_enable == True:
    shutil.copy(os.path.join("SIMD/", "SIMD_intrinsics.h"), os.path.join(Ccodesdir, "SIMD/"))
else:
    with open(os.path.join(Ccodesdir, "SIMD", "SIMD_intrinsics.h"), "w") as file:
        file.write("#define SIMD_IS_DISABLED\n")

# Set coordinate system and call the reference metric
CoordSystem = "Spherical"
par.set_parval_from_str("reference_metric::CoordSystem", CoordSystem)
rfm.reference_metric()

# Set domain size
domain_size = 10

# Set sinh width (for Sinh* coordinates)
sinh_width = 0.4

# Set boundary conditions type (QPE or Sommerfeld)
BC_Type = "Sommerfeld"

# Set finite differencing order
FD_order = 4
par.set_parval_from_str("finite_difference::FD_CENTDERIVS_ORDER", FD_order)

# Allow for the use of functions (instead of inline calculations) to compute finite differences
FD_functions_enable = True
par.set_parval_from_str("finite_difference::FD_functions_enable", FD_functions_enable)

# Set the core data type and the default CFL factor
REAL = "double"
default_CFL_FACTOR = 0.5

# Set mass and spin of the Black Hole
M = 1
chi = 0.99

# UIUC Black Hole ComputeADMGlobalsOnly
ComputeADMGlobalsOnly = True

# Toggle to evolve scalar field
scalar_field_evolve = False

# Set mass of the scalar field
mu_s = 0.00

# Set gaussian profile parameters for the scalar field
A  = 1.0
r0 = 32
w  = 8

## Generate Runge-Kutta timestepping code depending on Boundary Condition type

In [4]:
# Generate Runge-Kutta timestepping code
import MoLtimestepping.C_Code_Generation as MoL
from MoLtimestepping.RK_Butcher_Table_Dictionary import Butcher_dict

RK_method = "RK4"
RK_order = Butcher_dict[RK_method][1]
cmd.mkdir(os.path.join(Ccodesdir, "MoLtimestepping/"))

if BC_Type == "QPE":

    RHS_string = """
Ricci_eval(&rfmstruct, &params, RK_INPUT_GFS, auxevol_gfs);
rhs_eval(&rfmstruct, &params, auxevol_gfs, RK_INPUT_GFS, RK_OUTPUT_GFS);\n"""
    if scalar_field_evolve:
        RHS_string += "fields_rhs_eval(&rfmstruct, &params, RK_INPUT_GFS, RK_OUTPUT_GFS);\n"
    post_RHS_string = """
apply_bcs_curvilinear(&params, &bcstruct, NUM_EVOL_GFS, evol_gf_parity, RK_OUTPUT_GFS);
enforce_detgammabar_constraint(&rfmstruct, &params, RK_OUTPUT_GFS);\n"""

elif BC_Type == "Sommerfeld":

    RHS_string = """
Ricci_eval(&rfmstruct, &params, RK_INPUT_GFS, auxevol_gfs);
rhs_eval(&rfmstruct, &params, auxevol_gfs, RK_INPUT_GFS, RK_OUTPUT_GFS)\n;"""
    if scalar_field_evolve:
        RHS_string += "fields_rhs_eval(&rfmstruct, &params, RK_INPUT_GFS, RK_OUTPUT_GFS);\n"
    RHS_string += "apply_bcs_sommerfeld(&params, xx, &bcstruct, NUM_EVOL_GFS, evol_gf_parity, RK_INPUT_GFS, RK_OUTPUT_GFS);\n"
    post_RHS_string = """
apply_inner_parity_conditions(&params, &bcstruct, NUM_EVOL_GFS, evol_gf_parity, RK_OUTPUT_GFS);
enforce_detgammabar_constraint(&rfmstruct, &params, RK_OUTPUT_GFS);\n"""

MoL.MoL_C_Code_Generation(RK_method,
    RHS_string = RHS_string,
    post_RHS_string = post_RHS_string,
    outdir = os.path.join(Ccodesdir,"MoLtimestepping/"))

## Output find_timestep() function to C file

In [5]:
# Output the find_timestep() function to a C file
rfm.out_timestep_func_to_file(os.path.join(Ccodesdir, "find_timestep.h"))

## Define function to pickle the C function dict in the code generation below

In [6]:
# In the parallel C codegen below, the
def pickled_outC_function_dict(outC_function_dict):
    outstr = []
    outstr.append(pickle.dumps(len(outC_function_dict)))
    for Cfuncname, Cfunc in outC_function_dict.items():
        outstr.append(pickle.dumps(Cfuncname))
        outstr.append(pickle.dumps(Cfunc))
    return outstr

# Import initial data for the UIUC Black Hole

In [7]:
import BSSN.UIUCBlackHole as uiuc

def UIUCBlackHoleID():
    print("Generating optimised C code for UIUC Black Hole initial data. May take a while, depending on CoordSystem.")
    start = time.time()

    uiuc.UIUCBlackHole(ComputeADMGlobalsOnly=ComputeADMGlobalsOnly)    # Registers ID C function in dictionary, used below to output to file

    if ComputeADMGlobalsOnly:

        r = uiuc.r
        th = uiuc.th
        ph = uiuc.ph
        gammaSphDD = uiuc.gammaSphDD
        KSphDD = uiuc.KSphDD
        alphaSph = uiuc.alphaSph
        betaSphU = uiuc.betaSphU
        BSphU = uiuc.BSphU

        # Step 1: Set psi, the conformal factor:
        # Spin per unit mass
        a = M * chi

        # Defined under equation 1 in Liu, Etienne, & Shapiro (2009) https://arxiv.org/pdf/1001.4077.pdf
        # Boyer - Lindquist outer horizon
        rp = M + sp.sqrt(M**2 - a**2)
        # Boyer - Lindquist inner horizon
        rm = M - sp.sqrt(M**2 - a**2)

        # Boyer - Lindquist radius in terms of UIUC radius
        # Eq. 11
        # r_{BL} = r * ( 1 + r_+ / 4r )^2
        rBL = r * (1 + rp / (4 * r))**2

        # Expressions found below Eq. 2
        # Sigma = r_{BL}^2 + a^2 cos^2 theta
        SIG = rBL**2 + a**2 * sp.cos(th)**2

        # Delta = r_{BL}^2 - 2Mr_{BL} + a^2
        DEL = rBL**2 - 2 * M * rBL + a**2

        # A = (r_{BL}^2 + a^2)^2 - Delta a^2 sin^2 theta
        AA = (rBL**2 + a**2)**2 - DEL * a**2 * sp.sin(th)**2

        # Redefine alpha according to the original paper
        alphaSph = sp.sqrt(DEL * SIG / AA)

        # Redefine betaU according to the original paper
        betaSphU[2] = - 2 * a * M * rBL / AA

        import BSSN.ADM_Exact_Spherical_or_Cartesian_to_BSSNCurvilinear as AtoB
        Sph_r_th_ph = [r,th,ph]
        cf, hDD, lambdaU, aDD, trK, alpha, vetU, betU = \
            AtoB.Convert_Spherical_or_Cartesian_ADM_to_BSSN_curvilinear("Spherical", Sph_r_th_ph, gammaSphDD, KSphDD, alphaSph, betaSphU, BSphU)

        import BSSN.BSSN_ID_function_string as bIDf
        # Generates initial_data() C function & stores to outC_function_dict["initial_data"]
        bIDf.BSSN_ID_function_string(cf, hDD, lambdaU, aDD, trK, alpha, vetU, betU)
        
    with open(os.path.join(Ccodesdir, "initial_data.h"), "w") as file:
        file.write(outC_function_dict["initial_data"])
    end = time.time()
    print(f"(BENCH) Finished UIUC BH initial data codegen in {end - start} seconds.")
    return pickled_outC_function_dict(outC_function_dict)

# Output C Code for BSSN evolution of the spacetime

In [8]:
import BSSN.BSSN_RHSs as rhs
import BSSN.BSSN_gauge_RHSs as grhs
import BSSN.ScalarFieldRHSs as sfrhs

# Set the *covariant*, second-order Gamma-driving shift condition
par.set_parval_from_str("BSSN.BSSN_gauge_RHSs::ShiftEvolutionOption", "GammaDriving2ndOrder_Covariant")

print("Generating symbolic expressions for BSSN RHSs AND Scalar Field RHSs...")
start = time.time()

# Enable rfm_precompute to eliminate transcendental functions from BSSN RHSs
cmd.mkdir(os.path.join(Ccodesdir, "rfm_files/"))
par.set_parval_from_str("reference_metric::enable_rfm_precompute", "True")
par.set_parval_from_str("reference_metric::rfm_precompute_Ccode_outdir", os.path.join(Ccodesdir, "rfm_files/"))

# Evaluate BSSN + BSSN gauge RHSs with rfm_precompute enabled
import BSSN.BSSN_quantities as Bq
# Setting LeaveRicciSymbolic to True leads to a significantly shorter RHS symbolic generation,
# at the small cost of a two-step MoL algorithm to compute RHSs in the C code
par.set_parval_from_str("BSSN.BSSN_quantities::LeaveRicciSymbolic", "True")

rhs.BSSN_RHSs()
grhs.BSSN_gauge_RHSs()


# Generate symbolic expressions for the scalar field RHSs
if scalar_field_evolve:
    sfrhs.ScalarFieldRHSs()

# Use betaU as the upwinding control vector
Bq.BSSN_basic_tensors()
betaU = Bq.betaU

# Generate symbolic expressions to enforce the constraint on the determinant of gammabar
import BSSN.Enforce_Detgammabar_Constraint as EGC
enforce_detg_constraint_symb_expressions = EGC.Enforce_Detgammabar_Constraint_symb_expressions()

# Compute the Ricci tensor
par.set_parval_from_str("BSSN.BSSN_quantities::LeaveRicciSymbolic", "False")
Bq.RicciBar__gammabarDD_dHatD__DGammaUDD__DGammaU()

# Register the Hamiltonian constraint as a grid function
H = gri.register_gridfunctions("AUX", "H")
# Define the Hamiltonian constraint and ouptut the optimised C code
import BSSN.BSSN_constraints as bssncon
bssncon.BSSN_constraints(add_T4UUmunu_source_terms=False)

# Restore all rfm hatted quantities to their close-form expressions by setting rfm_precompute to False
par.set_parval_from_str("reference_metric::enable_rfm_precompute", "False")
rfm.ref_metric__hatted_quantities()

# Print elapsed time
end = time.time()
print(f"(BENCH) Finished BSSN  AND Scalar Field RHSs symbolic expression generation in {end - start} seconds.")

includes = None
if FD_functions_enable:
    includes = ["finite_difference_functions.h"]

Generating symbolic expressions for BSSN RHSs AND Scalar Field RHSs...
(BENCH) Finished BSSN  AND Scalar Field RHSs symbolic expression generation in 5.047461986541748 seconds.


In [9]:
# Function to output C code for computing the BSSN right-hand sides
def BSSN_RHSs():
    print(f"Generating C code for BSSN RHSs in {par.parval_from_str('reference_metric::CoordSystem')} coordinates.")
    start = time.time()

    # Construct the left-hand sides and right-hand side expressions for all BSSN RHSs
    lhs_names = [    "alpha",       "cf",       "trK"]
    rhs_exprs = [grhs.alpha_rhs, rhs.cf_rhs, rhs.trK_rhs]
    for i in range(3):
        lhs_names.append(   f"betU{i}")
        rhs_exprs.append(grhs.bet_rhsU[i])
        lhs_names.append(  f"lambdaU{i}")
        rhs_exprs.append(rhs.lambda_rhsU[i])
        lhs_names.append(   f"vetU{i}")
        rhs_exprs.append(grhs.vet_rhsU[i])
        for j in range(i, 3):
            lhs_names.append(f"aDD{i}{j}")
            rhs_exprs.append(rhs.a_rhsDD[i][j])
            lhs_names.append(f"hDD{i}{j}")
            rhs_exprs.append(rhs.h_rhsDD[i][j])

    # Sort the lhss list alphabetically, and the rhss to match
    lhs_names, rhs_exprs = [list(x) for x in zip(*sorted(zip(lhs_names, rhs_exprs), key=lambda pair: pair[0]))]

    # Declare the list of lhrh's
    BSSN_evol_rhss = []
    for var in range(len(lhs_names)):
        BSSN_evol_rhss.append(lhrh(lhs=gri.gfaccess("rhs_gfs", lhs_names[var]), rhs=rhs_exprs[var]))

    # Set up the C function for the BSSN RHSs
    # Set up the outputC and loop parameters for BSSN_RHSs C function
    outC_params = "outCverbose=False"
    loopoptions = "InteriorPoints,Enable_rfm_precompute"
    if SIMD_enable == True:
        loopoptions += ",EnableSIMD"
        outC_params += ",SIMD_enable=True"
    desc = "Evaluate the BSSN RHSs"
    name = "rhs_eval"
    outCfunction(
        outfile = os.path.join(Ccodesdir, name + ".h"), includes=includes, desc=desc, name=name,
        params = """rfm_struct *restrict rfmstruct, const paramstruct *restrict params,
                    const REAL *restrict auxevol_gfs, const REAL *restrict in_gfs, REAL *restrict rhs_gfs""",
        body = fin.FD_outputC("returnstring", BSSN_evol_rhss, params=outC_params, upwindcontrolvec=betaU).replace("IDX4", "IDX4S"),
        loopopts=loopoptions
    )

    # Print elapsed time
    end = time.time()
    print(f"(BENCH) Finished BSSN_RHS C codegen in {end - start} seconds.")
    return pickled_outC_function_dict(outC_function_dict)

In [10]:
# Function to output C code for computing the Ricci tensor
def Ricci():
    print(f"Generating C code for Ricci tensor in {par.parval_from_str('reference_metric::CoordSystem')} coordinates.")
    start = time.time()

    # Set up the C function for the Ricci tensor
    # Set outputC and loop parameters for Ricci tensor function
    outC_params = "outCverbose=False"
    loopoptions = "InteriorPoints,Enable_rfm_precompute"
    if SIMD_enable == True:
        loopoptions += ",EnableSIMD"
        outC_params += ",SIMD_enable=True"
    desc = "Evaluate the Ricci tensor"
    name = "Ricci_eval"
    outCfunction(
        outfile = os.path.join(Ccodesdir, name+".h"), includes=includes, desc=desc, name=name,
        params = """rfm_struct *restrict rfmstruct, const paramstruct *restrict params,
                    const REAL *restrict in_gfs, REAL *restrict auxevol_gfs""",
        body = fin.FD_outputC("returnstring",
                              [lhrh(lhs=gri.gfaccess("auxevol_gfs","RbarDD00"),rhs=Bq.RbarDD[0][0]),
                               lhrh(lhs=gri.gfaccess("auxevol_gfs","RbarDD01"),rhs=Bq.RbarDD[0][1]),
                               lhrh(lhs=gri.gfaccess("auxevol_gfs","RbarDD02"),rhs=Bq.RbarDD[0][2]),
                               lhrh(lhs=gri.gfaccess("auxevol_gfs","RbarDD11"),rhs=Bq.RbarDD[1][1]),
                               lhrh(lhs=gri.gfaccess("auxevol_gfs","RbarDD12"),rhs=Bq.RbarDD[1][2]),
                               lhrh(lhs=gri.gfaccess("auxevol_gfs","RbarDD22"),rhs=Bq.RbarDD[2][2])],
                               params=outC_params).replace("IDX4", "IDX4S"),
        loopopts=loopoptions
    )

    # Print elapsed time
    end = time.time()
    print(f"(BENCH) Finished Ricci C codegen in {end - start} seconds.")
    return pickled_outC_function_dict(outC_function_dict)

In [11]:
def ScalarField_RHSs():

    # Start timer
    print(f"Generating C code for Scalar Field RHSs in {par.parval_from_str('reference_metric::CoordSystem')} coordinates.")
    start = time.time()

    # Output to C code

    # Construct the LHSs and RHS expressions for the fields
    lhs_names = [      "Phi",         "Pi"]
    rhs_exprs = [ sfrhs.Phi_rhs, sfrhs.Pi_rhs]

    # Sort the lhss list alphabetically, and rhss to match.
    #   This ensures the RHSs are evaluated in the same order
    #   they're allocated in memory:
    # lhs_names, rhs_exprs = [list(x) for x in zip(*sorted(zip(lhs_names, rhs_exprs), key=lambda pair: pair[0]))]

    # Declare the list of lhrh's
    fields_rhss = []
    for var in range(len(lhs_names)):
        fields_rhss.append(lhrh(lhs=gri.gfaccess("rhs_gfs", lhs_names[var]), rhs=rhs_exprs[var]))

    # Set up the C function for evaluating the RHSs for the fields
    outC_params = "outCverbose=False"
    loopoptions = "InteriorPoints,Enable_rfm_precompute"
    if SIMD_enable == True:
        loopoptions += ",EnableSIMD"
        outC_params += ",SIMD_enable=True"
    desc = "Evaluate the scalar fields RHSs"
    name = "fields_rhs_eval"
    outCfunction(
        outfile = os.path.join(Ccodesdir, name + ".h"),
        desc = desc,
        name = name,
        params = "rfm_struct *restrict rfmstruct, const paramstruct *restrict params, const REAL *restrict in_gfs, REAL *restrict rhs_gfs",
        body = fin.FD_outputC("returnstring", fields_rhss, params=outC_params, upwindcontrolvec=betaU).replace("IDX4", "IDX4S"),
        loopopts = loopoptions
    )

    # End timer
    end = time.time()
    print(f'(BENCH) Finished Scalar Field RHSs C codegen in {end - start} seconds.')
    return pickled_outC_function_dict(outC_function_dict)

In [12]:
import BSSN.ScalarFieldID as sfid

# Generate initial data for the scalar fields
def ScalarField_ID():

    # Start timer
    start = time.time()
    print("Generating optimised C code for Scalar Field initial data. May take a while, depending on CoordSystem.")

    # Generate scalar field initial data expressions
    sfid.ScalarFieldID()

    # Print the expressions to a file
    with open(os.path.join(Ccodesdir, "fields_initial_data.h"), "w") as file:
        file.write(outC_function_dict["fields_initial_data"])

    # End timer
    end = time.time()
    print(f"(BENCH) Finished Scalar Field initial data codegen in {end - start} seconds.")
    return pickled_outC_function_dict(outC_function_dict)

In [13]:
# Function to output C code for computing the Hamiltonian constraint
def Hamiltonian():
    print("Generating optimised C code for the Hamiltonian constraint. May take a while, depending on CoordSystem.")
    start = time.time()

    # Set up the C function for the Hamiltonian RHS
    desc = "Evaluate the Hamiltonian constraint"
    name = "Hamiltonian_constraint"
    outCfunction(
        outfile = os.path.join(Ccodesdir, name + ".h"), includes=includes, desc=desc, name=name,
        params = """rfm_struct *restrict rfmstruct, const paramstruct *restrict params,
                    REAL *restrict in_gfs, REAL *restrict aux_gfs""",
        body = fin.FD_outputC("returnstring", lhrh(lhs=gri.gfaccess("aux_gfs", "H"), rhs=bssncon.H),
                               params="outCverbose=False").replace("IDX4", "IDX4S"),
        loopopts="InteriorPoints,Enable_rfm_precompute"
    )

    # Print elapsed time
    end = time.time()
    print(f"(BENCH) Finished Hamiltonian C codegen in {end - start} seconds.")
    return pickled_outC_function_dict(outC_function_dict)

In [14]:
# Function to output C code for enforcing the constraint on the determinant of gammabar
def gammadet():
    print("Generating optimised C code for gamma gonstraint. May take a while, depending on CoordSystem.")
    start = time.time()

    # Set up the C function for the det(gammahat) = det(gammabar)
    EGC.output_Enforce_Detgammabar_Constraint_Ccode(Ccodesdir, exprs=enforce_detg_constraint_symb_expressions)

    # Print elapsed time
    end = time.time()
    print(f"(BENCH) Finished gamma constraint C codegen in {end - start} seconds.")
    return pickled_outC_function_dict(outC_function_dict)

In [15]:
# Generate C code kernels for BSSN expressions, in parallel if possible

# Create a list of functions we wish to evaluate in parallel (if possible)
funcs = [UIUCBlackHoleID, BSSN_RHSs, Ricci, Hamiltonian, gammadet]
if scalar_field_evolve:
    funcs.append(ScalarField_RHSs)
    funcs.append(ScalarField_ID)
# pickled_outC_func_dict stores outC_function_dict from all the subprocesses in the parallel codegen
pickled_outC_func_dict = []

parallelise = True

try:
    if os.name == 'nt':
        # It's a mess to get working in Windows, so we don't bother. :/
        #  https://medium.com/@grvsinghal/speed-up-your-python-code-using-multiprocessing-on-windows-and-jupyter-or-ipython-2714b49d6fac
        raise Exception("Parallel codegen currently not available in certain environments, e.g., Windows")

    if not parallelise:
        raise Exception("Parallel codegen turned off. Proceeding to serival evaluation.")

    # Import the multiprocessing module.
    import multiprocessing as mp

    # Define master functions for parallelisation.
    def master_func(arg):
        return funcs[arg]()
    # Evaluate list of functions in parallel if possible
    # otherwise fallback to serial evaluation
    pool = mp.Pool()
    pickled_outC_func_dict.append(pool.map(master_func,range(len(funcs))))
    pool.terminate()
    pool.join()
except:
    # Fallback to serial evaluation if parallel not possible
    for func in funcs:
        func()
    pickled_outC_func_dict = []

# Output functions for computing all finite-difference stencils
if FD_functions_enable and len(pickled_outC_func_dict)>0:
    # First unpickle pickled_outC_func_dict
    outCfunc_dict = {}
    for WhichFunc in pickled_outC_func_dict[0]:
        i = 0
        num_elements = pickle.loads(WhichFunc[i]); i += 1
        for lst in range(num_elements):
            funcname = pickle.loads(WhichFunc[i])
            funcbody = pickle.loads(WhichFunc[i + 1]); i += 2
            outCfunc_dict[funcname] = funcbody
    # Then store the unpickled outCfunc_dict to outputC's outC_function_dict
    for key, item in outCfunc_dict.items():
        outC_function_dict[key] = item

if FD_functions_enable:
    # Finally generate finite_difference_functions.h
    fin.output_finite_difference_functions_h(path=Ccodesdir)

Generating optimised C code for the Hamiltonian constraint. May take a while, depending on CoordSystem.
Generating optimised C code for UIUC Black Hole initial data. May take a while, depending on CoordSystem.
Generating C code for BSSN RHSs in Spherical coordinates.
Generating C code for Ricci tensor in Spherical coordinates.
Generating optimised C code for gamma gonstraint. May take a while, depending on CoordSystem.
Output C function enforce_detgammabar_constraint() to file BSSN_LinearScalarFieldEvolution_Ccodes/enforce_detgammabar_constraint.h
(BENCH) Finished gamma constraint C codegen in 0.09948539733886719 seconds.
(BENCH) Finished UIUC BH initial data codegen in 1.0314700603485107 seconds.
Output C function rhs_eval() to file BSSN_LinearScalarFieldEvolution_Ccodes/rhs_eval.h
(BENCH) Finished BSSN_RHS C codegen in 12.476763486862183 seconds.
Output C function Ricci_eval() to file BSSN_LinearScalarFieldEvolution_Ccodes/Ricci_eval.h
(BENCH) Finished Ricci C codegen in 22.706571817

In [16]:
# Output C codes needed for declaring and setting Cparameters; also set free_parameters.h

# Generate declare_Cparameters_struct.h, set_Cparameters_default.h and set_Cparameters[-SIMD].h
par.generate_Cparameters_Ccodes(os.path.join(Ccodesdir))

# Set free_parameters.h
with open(os.path.join(Ccodesdir, "free_parameters.h"), "w") as file:
    file.write(f"""
// Set Black Hole mass and spin
params.M = {M};
params.chi = {chi};""")

    if scalar_field_evolve:
        file.write(f"""
// Set Scalar Field mass
params.mu_s = {mu_s};

// Set Scalar Field gaussian profile variables
params.A = {A};
params.r0 = {r0};
params.w = {w};
    """)

# Append to $Ccodesdir/free_parameters.h reference metric parameters based on generic
# domain_size,sinh_width,sinhv2_const_dr,SymTP_bScale, parameters set above.
rfm.out_default_free_parameters_for_rfm(os.path.join(Ccodesdir,"free_parameters.h"), domain_size, sinh_width)

# Generate set_Nxx_dxx_invdx_params__and__xx.h
rfm.set_Nxx_dxx_invdx_params__and__xx_h(Ccodesdir)

# Generate xxCart.h to map from xx to cartesian coordinates
rfm.xxCart_h("xxCart", "./set_Cparameters.h", os.path.join(Ccodesdir, "xxCart.h"))

# Generate declare_Cparameters_struct.h, set_Cparameters_default.h and set_Cparameters[-SIMD].h
par.generate_Cparameters_Ccodes(os.path.join(Ccodesdir))

# Set up boundary conditions

In [17]:
import CurviBoundaryConditions.CurviBoundaryConditions as cbcs
cbcs.Set_up_CurviBoundaryConditions(os.path.join(Ccodesdir,"boundary_conditions/"),Cparamspath=os.path.join("../"), BoundaryCondition=BC_Type)

if BC_Type == "Sommerfeld":
    bcs = cbcs.sommerfeld_bc(vars_radial_falloff_power_default=3., vars_speed_default=1., vars_at_inf_default=0.)

    # We could change values of GF speed or values at infinity, but the defaults work for now
    # Write values to parameter file
    bcs.write_sommerfeld_files(Ccodesdir, fd_order=2)

Wrote to file "BSSN_LinearScalarFieldEvolution_Ccodes/boundary_conditions/parity_conditions_symbolic_dot_products.h"
Evolved parity: ( aDD00:4, aDD01:5, aDD02:6, aDD11:7, aDD12:8, aDD22:9,
    alpha:0, betU0:1, betU1:2, betU2:3, cf:0, hDD00:4, hDD01:5, hDD02:6,
    hDD11:7, hDD12:8, hDD22:9, lambdaU0:1, lambdaU1:2, lambdaU2:3, trK:0,
    vetU0:1, vetU1:2, vetU2:3 )
Auxiliary parity: ( H:0 )
AuxEvol parity: ( RbarDD00:4, RbarDD01:5, RbarDD02:6, RbarDD11:7,
    RbarDD12:8, RbarDD22:9 )
Wrote to file "BSSN_LinearScalarFieldEvolution_Ccodes/boundary_conditions/EigenCoord_Cart_to_xx.h"


In [18]:
%%writefile $Ccodesdir/boundary_conditions/apply_inner_parity_conditions.h

void apply_inner_parity_conditions(const paramstruct *restrict params, const bc_struct *restrict bcstruct, const int NUM_GFS, const int8_t *restrict gfs_parity, REAL *restrict gfs) {

#include "../set_Cparameters.h"

#pragma omp parallel for
    for (int which_gf = 0; which_gf < NUM_GFS; which_gf++) {
        for (int which_gz = 0; which_gz < NGHOSTS; which_gz++) {
            for (int pt = 0; pt < bcstruct->num_ib_gz_pts[which_gz]; pt++)
            {
                const int i0dest = bcstruct->inner[which_gz][pt].inner_bc_dest_pt.i0;
                const int i1dest = bcstruct->inner[which_gz][pt].inner_bc_dest_pt.i1;
                const int i2dest = bcstruct->inner[which_gz][pt].inner_bc_dest_pt.i2;
                const int i0src = bcstruct->inner[which_gz][pt].inner_bc_src_pt.i0;
                const int i1src = bcstruct->inner[which_gz][pt].inner_bc_src_pt.i1;
                const int i2src = bcstruct->inner[which_gz][pt].inner_bc_src_pt.i2;
                const int8_t *prty = bcstruct->inner[which_gz][pt].parity;
                gfs[IDX4S(which_gf, i0dest, i1dest, i2dest)] = bcstruct->inner[which_gz][pt].parity[gfs_parity[which_gf]] * gfs[IDX4S(which_gf, i0src, i1src, i2src)];
            }
        }
    }
}


Writing BSSN_LinearScalarFieldEvolution_Ccodes//boundary_conditions/apply_inner_parity_conditions.h


# Main C Code

In [19]:
# Define REAL, set the number of ghost cells NGHOSTS (from NRPy+'s FD_CENTDERIVS_ORDER),
# and set the CFL_FACTOR (which can be overwritten at the command line)

with open(os.path.join(Ccodesdir,"BSSN_Playground_REAL__NGHOSTS__CFL_FACTOR.h"), "w") as file:
    file.write("""
// Part P0.a: Set the number of ghost cells, from NRPy+'s FD_CENTDERIVS_ORDER
#define NGHOSTS """+str(int(FD_order/2)+1)+"""
// Part P0.b: Set the numerical precision (REAL) to double, ensuring all floating point
//            numbers are stored to at least ~16 significant digits
#define REAL """+REAL+"""
// Part P0.c: Set the CFL Factor. Can be overwritten at command line.
REAL CFL_FACTOR = """+str(default_CFL_FACTOR)+";")

In [20]:
%%writefile $Ccodesdir/main.c

// Define REAL and NGHOSTS; and declare CFL_FACTOR. This header is generated in NRPy+.
#include "BSSN_Playground_REAL__NGHOSTS__CFL_FACTOR.h"

#include "rfm_files/rfm_struct__declare.h"

#include "declare_Cparameters_struct.h"

// All SIMD intrinsics used in SIMD-enabled C code loops are defined here:
#include "SIMD/SIMD_intrinsics.h"
#ifdef SIMD_IS_DISABLED
// Algorithm for upwinding, SIMD-disabled version.
// *NOTE*: This upwinding is backwards from
//  usual upwinding algorithms, because the
//  upwinding control vector in BSSN (the shift)
//  acts like a *negative* velocity.
#define UPWIND_ALG(UpwindVecU) UpwindVecU > 0.0 ? 1.0 : 0.0
#endif

// Import needed header files
#include <stdio.h>
#include <stdlib.h>
#include <math.h>
#include <time.h>
#include <stdint.h> // Needed for Windows GCC 6.x compatibility
#ifndef M_PI
#define M_PI 3.141592653589793238462643383279502884L
#endif
#ifndef M_SQRT1_2
#define M_SQRT1_2 0.707106781186547524400844362104849039L
#endif
#define wavespeed 1.0 // Set CFL-based "wavespeed" to 1.0.

// Declare the IDX4S(gf,i,j,k) macro, which enables us to store 4-dimensions of
// data in a 1D array. In this case, consecutive values of "i"
// (all other indices held to a fixed value) are consecutive in memory, where
// consecutive values of "j" (fixing all other indices) are separated by
// Nxx_plus_2NGHOSTS0 elements in memory. Similarly, consecutive values of
// "k" are separated by Nxx_plus_2NGHOSTS0*Nxx_plus_2NGHOSTS1 in memory, etc.
#define IDX4S(g, i, j, k) \
    ((i) + Nxx_plus_2NGHOSTS0 * ((j) + Nxx_plus_2NGHOSTS1 * ((k) + Nxx_plus_2NGHOSTS2 * (g))))
#define IDX4ptS(g, idx) ((idx) + (Nxx_plus_2NGHOSTS0 * Nxx_plus_2NGHOSTS1 * Nxx_plus_2NGHOSTS2) * (g))
#define IDX3S(i, j, k) ((i) + Nxx_plus_2NGHOSTS0 * ((j) + Nxx_plus_2NGHOSTS1 * ((k))))
#define LOOP_REGION(i0min, i0max, i1min, i1max, i2min, i2max) \
    for (int i2 = i2min; i2 < i2max; i2++)                    \
        for (int i1 = i1min; i1 < i1max; i1++)                \
            for (int i0 = i0min; i0 < i0max; i0++)
#define LOOP_ALL_GFS_GPS(ii) _Pragma("omp parallel for") for (int(ii) = 0; (ii) < Nxx_plus_2NGHOSTS_tot * NUM_EVOL_GFS; (ii)++)

// Set *GF macros, as well as xxCart()
#include "boundary_conditions/gridfunction_defines.h"

// Set xxCart(const paramstruct *restrict params,
//            REAL *restrict xx[3],
//            const int i0,const int i1,const int i2,
//            REAL xCart[3]),
// which maps xx->Cartesian via
// {xx[0][i0],xx[1][i1],xx[2][i2]}->{xCart[0],xCart[1],xCart[2]}
#include "xxCart.h"

// Defines set_Nxx_dxx_invdx_params__and__xx(const int EigenCoord, const int Nxx[3], paramstruct *restrict params, REAL *restrict xx[3]),
// which sets params Nxx,Nxx_plus_2NGHOSTS,dxx,invdx, and xx[] for
// the chosen Eigen-CoordSystem if EigenCoord==1, or
// CoordSystem if EigenCoord==0.
#include "set_Nxx_dxx_invdx_params__and__xx.h"

// Include basic functions needed to impose curvilinear
// parity and boundary conditions.
#include "boundary_conditions/CurviBC_include_Cfunctions.h"
#include "boundary_conditions/apply_inner_parity_conditions.h"

// Include function for enforcing detgammabar constraint.
#include "enforce_detgammabar_constraint.h"

// Find the CFL-constrained timestep
#include "find_timestep.h"

// Declare function necessary for setting up the initial data.
// Define BSSN_ID() for UIUC Black Hole initial data
// Set the generic driver function for setting up BSSN initial data
#include "initial_data.h"

// Declare and define function for setting up the initial data for the scalar field
#include "fields_initial_data.h"

// Declare function for evaluating Hamiltonian constraint (diagnostic)
#include "Hamiltonian_constraint.h"

// Declare rhs_eval function, which evaluates BSSN RHSs
#include "rhs_eval.h"

// Declare fields_rhs_eval function, which evaluates the scalar field RHSs
#include "fields_rhs_eval.h"

// Declare Ricci_eval function, which evaluates Ricci tensor
#include "Ricci_eval.h"

// main() function:
// Step 0: Read command-line input, set up grid structure, allocate memory for gridfunctions, set up coordinates
// Step 1: Set up initial data to an exact solution
// Step 2: Start the timer, for keeping track of how fast the simulation is progressing.
// Step 3: Integrate the initial data forward in time using the chosen RK-like Method of
//         Lines timestepping algorithm, and output periodic simulation diagnostics
// Step 3.a: Output 2D data file periodically, for visualization
// Step 3.b: Step forward one timestep (t -> t+dt) in time using
//           chosen RK-like MoL timestepping algorithm
// Step 3.c: If t=t_final, output conformal factor & Hamiltonian
//           constraint violation to 2D data file
// Step 3.d: Progress indicator printing to stderr
// Step 4: Free all allocated memory
int main(int argc, const char *argv[]) {

    paramstruct params;
#include "set_Cparameters_default.h"

    // Read command-line input, error out if nonconformant
    if ((argc != 4 && argc != 5) || atoi(argv[1]) < NGHOSTS || atoi(argv[2]) < NGHOSTS || atoi(argv[3]) < 2 /* FIXME; allow for axisymmetric sims */)
    {
        fprintf(stderr, "Error: Expected three command-line arguments: ./exec Nx0 Nx1 Nx2,\n");
        fprintf(stderr, "where Nx[0,1,2] is the number of grid points in the 0, 1, and 2 directions.\n");
        fprintf(stderr, "Nx[] MUST BE larger than NGHOSTS (= %d)\n", NGHOSTS);
        exit(1);
    }
    if (argc == 5)
    {
        CFL_FACTOR = strtod(argv[4], NULL);
        if (CFL_FACTOR > 0.5 && atoi(argv[3]) != 2)
        {
            fprintf(stderr, "WARNING: CFL_FACTOR was set to %e, which is > 0.5.\n", CFL_FACTOR);
            fprintf(stderr, "         This will generally only be stable if the simulation is purely axisymmetric\n");
            fprintf(stderr, "         However, Nx2 was set to %d>2, which implies a non-axisymmetric simulation\n", atoi(argv[3]));
        }
    }
    // Set up numerical grid structure, first in space...
    const int Nxx[3] = {atoi(argv[1]), atoi(argv[2]), atoi(argv[3])};
    if (Nxx[0] % 2 != 0 || Nxx[1] % 2 != 0 || Nxx[2] % 2 != 0)
    {
        fprintf(stderr, "Error: Cannot guarantee a proper cell-centered grid if number of grid cells not set to even number.\n");
        fprintf(stderr, "       For example, in case of angular directions, proper symmetry zones will not exist.\n");
        exit(1);
    }

    // Set free parameters, overwriting Cparameters defaults
    //          by hand or with command-line input, as desired.
#include "free_parameters.h"

    // Uniform coordinate grids are stored to *xx[3]
    REAL *xx[3];
    // Set bcstruct
    bc_struct bcstruct;
    {
        int EigenCoord = 1;
        // Call set_Nxx_dxx_invdx_params__and__xx(), which sets
        // params Nxx,Nxx_plus_2NGHOSTS,dxx,invdx, and xx[] for the
        // chosen Eigen-CoordSystem.
        set_Nxx_dxx_invdx_params__and__xx(EigenCoord, Nxx, &params, xx);
        // Set Nxx_plus_2NGHOSTS_tot
#include "set_Cparameters-nopointer.h"
        const int Nxx_plus_2NGHOSTS_tot = Nxx_plus_2NGHOSTS0 * Nxx_plus_2NGHOSTS1 * Nxx_plus_2NGHOSTS2;
        // Find ghostzone mappings; set up bcstruct
#include "boundary_conditions/driver_bcstruct.h"
        // Free allocated space for xx[][] array
        for (int i = 0; i < 3; i++)
            free(xx[i]);
    }

    // Call set_Nxx_dxx_invdx_params__and__xx(), which sets
    // params Nxx,Nxx_plus_2NGHOSTS,dxx,invdx, and xx[] for the
    // chosen (non-Eigen) CoordSystem.
    int EigenCoord = 0;
    set_Nxx_dxx_invdx_params__and__xx(EigenCoord, Nxx, &params, xx);

    // Set all C parameters "blah" for params.blah, including
    // Nxx_plus_2NGHOSTS0 = params.Nxx_plus_2NGHOSTS0, etc.
#include "set_Cparameters-nopointer.h"
    const int Nxx_plus_2NGHOSTS_tot = Nxx_plus_2NGHOSTS0 * Nxx_plus_2NGHOSTS1 * Nxx_plus_2NGHOSTS2;

    // Time coordinate parameters
    const REAL t_final = 100.0; /* Final time is set so that at t=t_final,
                                        * data at the origin have not been corrupted
                                        * by the approximate outer boundary condition */

    // Set timestep based on smallest proper distance between gridpoints and CFL factor
    REAL dt = find_timestep(&params, xx);
    //fprintf(stderr,"# Timestep set to = %e\n",(double)dt);
    int N_final = (int)(t_final / dt + 0.5); // The number of points in time.
                                             // Add 0.5 to account for C rounding down
                                             // typecasts to integers.
    int output_every_N = (int)((REAL)N_final / 50.0);
    if (output_every_N == 0)
        output_every_N = 1;

    // Error out if the number of auxiliary gridfunctions outnumber evolved gridfunctions.
    // This is a limitation of the RK method. You are always welcome to declare & allocate
    // additional gridfunctions by hand.
    if (NUM_AUX_GFS > NUM_EVOL_GFS)
    {
        fprintf(stderr, "Error: NUM_AUX_GFS > NUM_EVOL_GFS. Either reduce the number of auxiliary gridfunctions,\n");
        fprintf(stderr, "       or allocate (malloc) by hand storage for *diagnostic_output_gfs. \n");
        exit(1);
    }

    // Allocate memory for gridfunctions
#include "MoLtimestepping/RK_Allocate_Memory.h"
    REAL *restrict auxevol_gfs = (REAL *)malloc(sizeof(REAL) * NUM_AUXEVOL_GFS * Nxx_plus_2NGHOSTS_tot);

    // Set up precomputed reference metric arrays
    // Allocate space for precomputed reference metric arrays.
#include "rfm_files/rfm_struct__malloc.h"

    // Define precomputed reference metric arrays.
    {
#include "set_Cparameters-nopointer.h"
#include "rfm_files/rfm_struct__define.h"
    }

    // Set up initial data to an exact solution
    initial_data(&params, xx, y_n_gfs);
    fields_initial_data(&params, xx, y_n_gfs);

    // Apply inner parity conditions, as initial data
    // are sometimes ill-defined in ghost zones.
    // E.g., spherical initial data might not be
    // properly defined at points where r=-1.
    apply_inner_parity_conditions(&params, &bcstruct, NUM_EVOL_GFS, evol_gf_parity, y_n_gfs);
    enforce_detgammabar_constraint(&rfmstruct, &params, y_n_gfs);

    // Start the timer, for keeping track of how fast the simulation is progressing.
#ifdef __linux__ // Use high-precision timer in Linux.
    struct timespec start, end;
    clock_gettime(CLOCK_REALTIME, &start);
#else // Resort to low-resolution, standards-compliant timer in non-Linux OSs
    // http://www.cplusplus.com/reference/ctime/time/
    time_t start_timer, end_timer;
    time(&start_timer); // Resolution of one second...
#endif

    // Integrate the initial data forward in time using the chosen RK-like Method of
    // Lines timestepping algorithm, and output periodic simulation diagnostics
    for (int n = 0; n <= N_final; n++)
    { // Main loop to progress forward in time.

        // Output 2D data file periodically, for visualization
        if (n % output_every_N == 0)
        {
            // Evaluate Hamiltonian constraint violation
            Hamiltonian_constraint(&rfmstruct, &params, y_n_gfs, diagnostic_output_gfs);

            char filename[100];
            sprintf(filename, "out%d-%08d.txt", Nxx[0], n);
            FILE *out = fopen(filename, "w");
            LOOP_REGION(NGHOSTS, Nxx_plus_2NGHOSTS0 - NGHOSTS,
                        NGHOSTS, Nxx_plus_2NGHOSTS1 - NGHOSTS,
                        NGHOSTS, Nxx_plus_2NGHOSTS2 - NGHOSTS)
            {
                const int idx = IDX3S(i0, i1, i2);
                REAL xx0 = xx[0][i0];
                REAL xx1 = xx[1][i1];
                REAL xx2 = xx[2][i2];
                fprintf(out, "%e %e %e %e %e %e %e\n",
                        xx0, xx1, xx2, y_n_gfs[IDX4ptS(CFGF, idx)], diagnostic_output_gfs[IDX4ptS(HGF, idx)],
                        y_n_gfs[IDX4ptS(PHIGF, idx)], y_n_gfs[IDX4ptS(PIGF, idx)]);
            }

            fclose(out);
            }

        // Step forward one timestep (t -> t+dt) in time using
        // chosen RK-like MoL timestepping algorithm
#include "MoLtimestepping/RK_MoL.h"

        // If t=t_final, output conformal factor & Hamiltonian
        // constraint violation to 2D data file
        if (n == N_final - 1)
        {
            // Evaluate Hamiltonian constraint violation
            Hamiltonian_constraint(&rfmstruct, &params, y_n_gfs, diagnostic_output_gfs);

            char filename[100];
            sprintf(filename, "out%d-%08d.txt", Nxx[0], n);
            FILE *out = fopen(filename, "w");
            LOOP_REGION(NGHOSTS, Nxx_plus_2NGHOSTS0 - NGHOSTS,
                        NGHOSTS, Nxx_plus_2NGHOSTS1 - NGHOSTS,
                        NGHOSTS, Nxx_plus_2NGHOSTS2 - NGHOSTS)
            {
                const int idx = IDX3S(i0, i1, i2);
                REAL xx0 = xx[0][i0];
                REAL xx1 = xx[1][i1];
                REAL xx2 = xx[2][i2];
                fprintf(out, "%e %e %e %e %e %e %e\n",
                        xx0, xx1, xx2, y_n_gfs[IDX4ptS(CFGF, idx)], diagnostic_output_gfs[IDX4ptS(HGF, idx)],
                        y_n_gfs[IDX4ptS(PHIGF, idx)], y_n_gfs[IDX4ptS(PIGF, idx)]);
            }

            fclose(out);
        }
        // Progress indicator printing to stderr

        // Measure average time per iteration
#ifdef __linux__ // Use high-precision timer in Linux.
        clock_gettime(CLOCK_REALTIME, &end);
        const long long unsigned int time_in_ns = 1000000000L * (end.tv_sec - start.tv_sec) + end.tv_nsec - start.tv_nsec;
#else // Resort to low-resolution, standards-compliant timer in non-Linux OSs
        time(&end_timer);                                                 // Resolution of one second...
        REAL time_in_ns = difftime(end_timer, start_timer) * 1.0e9 + 0.5; // Round up to avoid divide-by-zero.
#endif
        const REAL s_per_iteration_avg = ((REAL)time_in_ns / (REAL)n) / 1.0e9;

        const int iterations_remaining = N_final - n;
        const REAL time_remaining_in_mins = s_per_iteration_avg * (REAL)iterations_remaining / 60.0;

        const REAL num_RHS_pt_evals = (REAL)(Nxx[0] * Nxx[1] * Nxx[2]) * 4.0 * (REAL)n; // 4 RHS evals per gridpoint for RK4
        const REAL RHS_pt_evals_per_sec = num_RHS_pt_evals / ((REAL)time_in_ns / 1.0e9);

        // Output simulation progress to stderr
        if (n % 10 == 0)
        {
            fprintf(stderr, "%c[2K", 27);                                                          // Clear the line
            fprintf(stderr, "It: %d t=%.2f dt=%.2e | %.1f%%; ETA %.0f s | t/h %.2f | gp/s %.2e\r", // \r is carriage return, move cursor to the beginning of the line
                    n, n * (double)dt, (double)dt, (double)(100.0 * (REAL)n / (REAL)N_final),
                    (double)time_remaining_in_mins * 60, (double)(dt * 3600.0 / s_per_iteration_avg), (double)RHS_pt_evals_per_sec);
            fflush(stderr); // Flush the stderr buffer
        }                   // End progress indicator if(n % 10 == 0)
    }                       // End main loop to progress forward in time.
    fprintf(stderr, "\n");  // Clear the final line of output from progress indicator.

    // Free all allocated memory
#include "rfm_files/rfm_struct__freemem.h"
#include "boundary_conditions/bcstruct_freemem.h"
#include "MoLtimestepping/RK_Free_Memory.h"
    free(auxevol_gfs);
    for (int i = 0; i < 3; i++)
        free(xx[i]);

    return 0;
}

Writing BSSN_LinearScalarFieldEvolution_Ccodes//main.c
